In [5]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates

# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

## Data reading and melting

In [6]:
data = pd.read_csv("Adjusted Clos_collated.csv").drop("Unnamed: 0", axis = 1)

In [7]:
data

,AsOnDate,20 Microns Ltd.,20Th Century Finance Corpn. Ltd. [Merged],360 One Wam Ltd.,3I Infotech Ltd.,3M India Ltd.,3P Land Holdings Ltd.,3Rd Rock Multimedia Ltd.,5Paisa Capital Ltd.,63 Moons Technologies Ltd.,...,3898,3899,3900,3901,3902,3903,3904,3905,3906,3907
0,2004-01-01,NaN,NaN,NaN,NaN,515.00,NaN,NaN,NaN,NaN,...,66.83,NaN,15.25,NaN,NaN,NaN,59.85,25.79,NaN,NaN
1,2004-01-02,NaN,NaN,NaN,NaN,530.50,3.20,NaN,NaN,NaN,...,65.88,NaN,17.35,NaN,NaN,NaN,66.50,26.74,NaN,NaN
2,2004-01-05,NaN,NaN,NaN,NaN,511.75,3.42,NaN,NaN,NaN,...,61.76,NaN,16.30,NaN,NaN,NaN,62.70,25.95,NaN,NaN
3,2004-01-06,NaN,NaN,NaN,NaN,495.00,NaN,NaN,NaN,NaN,...,61.94,NaN,15.70,NaN,NaN,NaN,59.50,25.74,NaN,NaN
4,2004-01-07,NaN,NaN,NaN,NaN,490.40,2.76,NaN,NaN,NaN,...,60.33,NaN,16.00,NaN,NaN,NaN,57.75,25.15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5025,2024-03-21,144.70,12.5,673.40,41.60,30049.25,30.80,64.6,487.45,407.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5026,2024-03-22,145.05,12.5,665.85,41.85,30728.40,30.40,64.6,487.85,402.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5027,2024-03-26,143.00,12.5,650.30,39.20,30487.15,28.90,64.6,476.05,393.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5028,2024-03-27,143.00,12.5,667.95,39.40,31475.10,28.40,64.6,490.30,381.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dataLong = data.melt( id_vars = "AsOnDate", value_vars = data.columns[1:3908]).rename({"value":"ACP", "variable":"CompanyName"}, axis = 1).drop_duplicates().reset_index(drop = True)
dataLong = dataLong.loc[~( (dataLong.duplicated(subset = ["CompanyName", "AsOnDate"], keep = False)) & (dataLong["ACP"].isnull())) ]
dataLong = dataLong.loc[~ dataLong.duplicated(subset = ["CompanyName", "AsOnDate"], keep = False)].drop_duplicates().reset_index(drop = True)

In [9]:
dataLong

,AsOnDate,CompanyName,ACP
0,2004-01-01,20 Microns Ltd.,NaN
1,2004-01-02,20 Microns Ltd.,NaN
2,2004-01-05,20 Microns Ltd.,NaN
3,2004-01-06,20 Microns Ltd.,NaN
4,2004-01-07,20 Microns Ltd.,NaN
...,...,...,...
19640110,2024-03-21,3907,NaN
19640111,2024-03-22,3907,NaN
19640112,2024-03-26,3907,NaN
19640113,2024-03-27,3907,NaN


## Data Cleaning

### Data Snipping from either ends

In [10]:
def dataSnip(frame):
    
    first_valid_idx = frame["ACP"].first_valid_index()
    
    if first_valid_idx is not None:
        frame = frame.loc[first_valid_idx:]
        
    else:
        frame = frame.iloc[0:0]

    last_valid_idx = frame["ACP"].last_valid_index()
    
    if last_valid_idx is not None:
        frame = frame.loc[:last_valid_idx]
        
    else:
        frame = frame
        
    return frame

In [11]:
dataLong2 = dataLong.groupby(by="CompanyName").progress_apply(dataSnip).reset_index(drop=True)
dataLong2

  0%|          | 0/3907 [00:00<?, ?it/s]

,AsOnDate,CompanyName,ACP
0,2008-10-06,20 Microns Ltd.,16.82
1,2008-10-07,20 Microns Ltd.,15.05
2,2008-10-08,20 Microns Ltd.,13.25
3,2008-10-10,20 Microns Ltd.,11.60
4,2008-10-13,20 Microns Ltd.,12.32
...,...,...,...
13465088,2024-03-21,Zylog Systems Ltd.,0.35
13465089,2024-03-22,Zylog Systems Ltd.,0.35
13465090,2024-03-26,Zylog Systems Ltd.,0.35
13465091,2024-03-27,Zylog Systems Ltd.,0.35


### Inter series NaN values filled as the previous value

In [12]:
def dataForwardFill(frame):

    frame["ACP"] = frame["ACP"].ffill()
        
    return frame

In [13]:
dataLong3 = dataLong2.groupby(by="CompanyName").apply(dataForwardFill).reset_index(drop=True)
dataLong3

,AsOnDate,CompanyName,ACP
0,2008-10-06,20 Microns Ltd.,16.82
1,2008-10-07,20 Microns Ltd.,15.05
2,2008-10-08,20 Microns Ltd.,13.25
3,2008-10-10,20 Microns Ltd.,11.60
4,2008-10-13,20 Microns Ltd.,12.32
...,...,...,...
13465088,2024-03-21,Zylog Systems Ltd.,0.35
13465089,2024-03-22,Zylog Systems Ltd.,0.35
13465090,2024-03-26,Zylog Systems Ltd.,0.35
13465091,2024-03-27,Zylog Systems Ltd.,0.35


### Result

In [14]:
dataLong3.ACP.isnull().value_counts()

ACP
False    13465093
Name: count, dtype: int64

In [15]:
closingPrice = dataLong3.copy()

## Simple Returns

In [16]:
def pct_change(frame):
    frame = frame.sort_values(by = ["AsOnDate"])
    frame["pct"] = frame["ACP"].pct_change(fill_method = None)
    return frame

In [17]:
simpleReturn = closingPrice.groupby("CompanyName").apply(pct_change).reset_index(drop = True)

In [18]:
simpleReturn["AsOnDate"] = pd.to_datetime(simpleReturn["AsOnDate"], format = "%Y-%m-%d")
simpleReturn = simpleReturn.drop_duplicates().reset_index(drop = True)
simpleReturn

,AsOnDate,CompanyName,ACP,pct
0,2008-10-06,20 Microns Ltd.,16.82,NaN
1,2008-10-07,20 Microns Ltd.,15.05,-0.105232
2,2008-10-08,20 Microns Ltd.,13.25,-0.119601
3,2008-10-10,20 Microns Ltd.,11.60,-0.124528
4,2008-10-13,20 Microns Ltd.,12.32,0.062069
...,...,...,...,...
13465088,2024-03-21,Zylog Systems Ltd.,0.35,0.000000
13465089,2024-03-22,Zylog Systems Ltd.,0.35,0.000000
13465090,2024-03-26,Zylog Systems Ltd.,0.35,0.000000
13465091,2024-03-27,Zylog Systems Ltd.,0.35,0.000000


## FF 3 constants

In [19]:
ff3Const = pd.read_csv("2024-03_FourFactors_and_Market_Returns_Daily_SurvivorshipBiasAdjusted.csv")

In [20]:
ff3Const["date"] = pd.to_datetime(ff3Const["date"], format = "%d-%m-%Y")

In [21]:
ff3Const = ff3Const.rename({"date":"AsOnDate"}, axis = 1).drop_duplicates().reset_index(drop = True)
ff3Const["RMRF"] = ff3Const["MF"] - ff3Const["RF"]
ff3Const.columns[[0,1,2,3,4,5,6]]
orderedCol = ff3Const.columns[[0, 5, 6, 4, 1, 2]]
ff3ConstOrdered = ff3Const[orderedCol]
ff3ConstOrdered

,AsOnDate,RF,RMRF,MF,SMB,HML
0,1993-10-01,NaN,NaN,NaN,1.414154,1.182552
1,1993-10-04,0.022014,-0.954330,-0.932316,0.472301,0.371959
2,1993-10-05,0.022014,-0.315512,-0.293498,0.046619,0.839734
3,1993-10-06,0.022014,-0.352836,-0.330823,-0.085561,-1.492747
4,1993-10-07,0.022014,0.360946,0.382959,-0.286668,0.135259
...,...,...,...,...,...,...
7563,2024-03-21,0.018215,1.441807,1.460021,0.572866,1.270303
7564,2024-03-22,0.018215,0.593324,0.611538,0.752002,0.124527
7565,2024-03-26,0.072878,-0.044514,0.028364,-1.091597,0.440605
7566,2024-03-27,0.018215,0.326702,0.344916,-0.165072,-0.188505


## Merged and Final Data set

In [22]:
dataMerged = simpleReturn.merge(ff3ConstOrdered, on = "AsOnDate", how = "inner").sort_values(["CompanyName", "AsOnDate"]).set_index(["CompanyName", "AsOnDate"]).reset_index().drop_duplicates().reset_index(drop = True)

In [23]:
dataMerged

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853
...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505


## Event Studies Package

In [24]:
# es.Single.import_FamaFrench("2024-03_FourFactors_and_Market_Returns_Daily.csv", rescale_factor = False, date_format = "%d-%m-%Y")

# Organic Functions

In [25]:
# Just Trying
# FIT OLS, with R style formulas
# res = smf.ols("pct - RF ~ RMRF + SMB + HML", data = dataMerged).fit()

In [26]:
# res.summary()

In [27]:
dataMerged.CompanyName.nunique()

3721

## 120 Day Event Study

### 120 day window OLS

In [106]:
# pre-event 120 days
# ignoring companies with <130 data points in full.

def OLS120(frame):
    frame = frame.set_index("AsOnDate")
    if len(frame) > 131:
        exog_vars = ["RMRF", "SMB", "HML"]
        endog = frame.pct - frame.RF
        exog = sm.add_constant(frame[exog_vars])
        rols = RollingOLS(endog, exog, window = 120)
        res = rols.fit()
        
        outputFrame = res.params
        
        # outputFrame["OLS120_intercept_p_value"] = [item[0] for item in res.pvalues]
        # outputFrame["OLS120_RMRF_p_value"] = [item[1] for item in res.pvalues]
        # outputFrame["OLS120_SMB_p_value"] = [item[2] for item in res.pvalues]
        # outputFrame["OLS120_HML_p_value"] = [item[3] for item in res.pvalues]
        
        # outputFrame["OLS120_intercept_t_value"] = res.tvalues.const
        # outputFrame["OLS120_RMRF_t_value"] = res.tvalues.RMRF
        # outputFrame["OLS120_SMB_t_value"] = res.tvalues.SMB
        # outputFrame["OLS120_HML_t_value"] = res.tvalues.HML

        outputFrame["OLS120_r_squared"] = res.rsquared
        outputFrame["OLS120_adjusted_r_squared"] = res.rsquared_adj
        outputFrame["OLS120_f_p_value"] = res.f_pvalue
        
        return outputFrame.shift(periods = 11).dropna(subset = "const")

In [107]:
ols120Param = dataMerged.groupby(by = ["CompanyName"]).progress_apply(OLS120).reset_index()
ols120Param = ols120Param.rename({"const":"OLS120_intercept", "RMRF":"OLS120_RMRF", "SMB":"OLS120_SMB", "HML":"OLS120_HML"}, axis = 1)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [108]:
ols120Param

,CompanyName,AsOnDate,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value
0,20 Microns Ltd.,2009-04-27,-0.024856,0.012905,0.011293,0.001436,0.274365,0.255435,4.519505e-08
1,20 Microns Ltd.,2009-04-28,-0.024314,0.013002,0.011581,0.002395,0.275674,0.256942,3.488935e-08
2,20 Microns Ltd.,2009-04-29,-0.023559,0.012881,0.011145,0.003962,0.282966,0.264422,1.964126e-08
3,20 Microns Ltd.,2009-05-04,-0.024365,0.011572,0.008292,0.005490,0.248402,0.228964,2.834241e-07
4,20 Microns Ltd.,2009-05-05,-0.023842,0.010647,0.007331,0.007090,0.218311,0.198095,2.600825e-06
...,...,...,...,...,...,...,...,...,...
12989588,Zylog Systems Ltd.,2024-03-21,-0.026702,0.003907,0.003905,-0.001484,0.049581,0.025001,1.153618e-01
12989589,Zylog Systems Ltd.,2024-03-22,-0.026628,0.004073,0.003156,-0.001910,0.044392,0.019678,1.517656e-01
12989590,Zylog Systems Ltd.,2024-03-26,-0.026633,0.004079,0.003177,-0.001971,0.045188,0.020494,1.455639e-01
12989591,Zylog Systems Ltd.,2024-03-27,-0.026979,0.004395,0.004305,-0.001647,0.062475,0.038228,5.716690e-02


In [109]:
ols120 = dataMerged.merge(ols120Param, on = ["CompanyName", "AsOnDate"], how = "left")

In [111]:
ols120

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS120_intercept,OLS120_RMRF,OLS120_SMB,OLS120_HML,OLS120_r_squared,OLS120_adjusted_r_squared,OLS120_f_p_value
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026702,0.003907,0.003905,-0.001484,0.049581,0.025001,0.115362
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026628,0.004073,0.003156,-0.001910,0.044392,0.019678,0.151766
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026633,0.004079,0.003177,-0.001971,0.045188,0.020494,0.145564
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026979,0.004395,0.004305,-0.001647,0.062475,0.038228,0.057167


In [112]:
ols120.to_csv("ols120.csv")

## 150 Day Event Study

### 150 day window OLS

In [ ]:
# pre-event 150 days
# ignoring companies with <130 data points in full.

def OLS150(frame):
    frame = frame.set_index("AsOnDate")
    if len(frame) > 161:
        exog_vars = ["RMRF", "SMB", "HML"]
        endog = frame.pct - frame.RF
        exog = sm.add_constant(frame[exog_vars])
        rols = RollingOLS(endog, exog, window = 150)
        res = rols.fit()
        
        outputFrame = res.params
        
        # outputFrame["OLS150_intercept_p_value"] = [item[0] for item in res.pvalues]
        # outputFrame["OLS150_RMRF_p_value"] = [item[1] for item in res.pvalues]
        # outputFrame["OLS150_SMB_p_value"] = [item[2] for item in res.pvalues]
        # outputFrame["OLS150_HML_p_value"] = [item[3] for item in res.pvalues]
        
        # outputFrame["OLS150_intercept_t_value"] = res.tvalues.const
        # outputFrame["OLS150_RMRF_t_value"] = res.tvalues.RMRF
        # outputFrame["OLS150_SMB_t_value"] = res.tvalues.SMB
        # outputFrame["OLS150_HML_t_value"] = res.tvalues.HML

        outputFrame["OLS150_r_squared"] = res.rsquared
        outputFrame["OLS150_adjusted_r_squared"] = res.rsquared_adj
        outputFrame["OLS150_f_p_value"] = res.f_pvalue
        
        return outputFrame.shift(periods = 11).dropna(subset = "const")

In [ ]:
ols150Param = dataMerged.groupby(by = ["CompanyName"]).progress_apply(OLS150).reset_index()
ols150Param = ols150Param.rename({"const":"OLS150_intercept", "RMRF":"OLS150_RMRF", "SMB":"OLS150_SMB", "HML":"OLS150_HML"}, axis = 1)

In [ ]:
ols150Param

In [ ]:
ols150 = dataMerged.merge(ols150Param, on = ["CompanyName", "AsOnDate"], how = "left")

In [ ]:
ols150

In [ ]:
ols150.to_csv("ols150.csv")

## 180 Day Event Study

### 180 day window OLS

In [28]:
# pre-event 180 days
# ignoring companies with <130 data points in full.

def OLS180(frame):
    frame = frame.set_index("AsOnDate")
    if len(frame) > 191:
        exog_vars = ["RMRF", "SMB", "HML"]
        endog = frame.pct - frame.RF
        exog = sm.add_constant(frame[exog_vars])
        rols = RollingOLS(endog, exog, window = 180)
        res = rols.fit()
        
        outputFrame = res.params
        
        # outputFrame["OLS180_intercept_p_value"] = [item[0] for item in res.pvalues]
        # outputFrame["OLS180_RMRF_p_value"] = [item[1] for item in res.pvalues]
        # outputFrame["OLS180_SMB_p_value"] = [item[2] for item in res.pvalues]
        # outputFrame["OLS180_HML_p_value"] = [item[3] for item in res.pvalues]
        
        # outputFrame["OLS180_intercept_t_value"] = res.tvalues.const
        # outputFrame["OLS180_RMRF_t_value"] = res.tvalues.RMRF
        # outputFrame["OLS180_SMB_t_value"] = res.tvalues.SMB
        # outputFrame["OLS180_HML_t_value"] = res.tvalues.HML

        outputFrame["OLS180_r_squared"] = res.rsquared
        outputFrame["OLS180_adjusted_r_squared"] = res.rsquared_adj
        outputFrame["OLS180_f_p_value"] = res.f_pvalue

        return outputFrame.shift(periods = 11).dropna(subset = "const")

In [29]:
ols180Param = dataMerged.groupby(by = ["CompanyName"]).progress_apply(OLS180).reset_index()
ols180Param = ols180Param.rename({"const":"OLS180_intercept", "RMRF":"OLS180_RMRF", "SMB":"OLS180_SMB", "HML":"OLS180_HML"}, axis = 1)

  0%|          | 0/3721 [00:00<?, ?it/s]

In [30]:
ols180Param

,CompanyName,AsOnDate,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value
0,20 Microns Ltd.,2009-07-22,-0.021264,0.009618,0.012246,0.004820,0.191707,0.177851,3.880499e-08
1,20 Microns Ltd.,2009-07-23,-0.021177,0.009572,0.012166,0.004828,0.191666,0.177887,3.513662e-08
2,20 Microns Ltd.,2009-07-24,-0.020713,0.009391,0.011592,0.005322,0.187514,0.173665,5.458318e-08
3,20 Microns Ltd.,2009-07-27,-0.020111,0.008978,0.010864,0.005588,0.171890,0.157774,2.800550e-07
4,20 Microns Ltd.,2009-07-28,-0.019722,0.008387,0.010366,0.007027,0.157005,0.142635,1.287276e-06
...,...,...,...,...,...,...,...,...,...
12776135,Zylog Systems Ltd.,2024-03-21,-0.026511,0.002663,0.002803,-0.002113,0.025498,0.008887,2.071184e-01
12776136,Zylog Systems Ltd.,2024-03-22,-0.026262,0.002406,0.002773,-0.002086,0.024682,0.008058,2.203633e-01
12776137,Zylog Systems Ltd.,2024-03-26,-0.026266,0.002386,0.002795,-0.002064,0.024686,0.008062,2.202977e-01
12776138,Zylog Systems Ltd.,2024-03-27,-0.026598,0.002727,0.003744,-0.001882,0.035403,0.018961,9.527629e-02


In [31]:
ols180 = dataMerged.merge(ols180Param, on = ["CompanyName", "AsOnDate"], how = "left")

In [32]:
ols180

,CompanyName,AsOnDate,ACP,pct,RF,RMRF,MF,SMB,HML,OLS180_intercept,OLS180_RMRF,OLS180_SMB,OLS180_HML,OLS180_r_squared,OLS180_adjusted_r_squared,OLS180_f_p_value
0,20 Microns Ltd.,2008-10-06,16.82,NaN,0.069713,-6.381449,-6.311735,-0.373052,-0.566450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20 Microns Ltd.,2008-10-07,15.05,-0.105232,0.023232,-0.669144,-0.645911,-1.502487,0.184699,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20 Microns Ltd.,2008-10-08,13.25,-0.119601,0.023232,-3.533362,-3.510130,-1.780674,0.072932,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20 Microns Ltd.,2008-10-10,11.60,-0.124528,0.045520,-7.052324,-7.006804,0.217126,0.354629,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20 Microns Ltd.,2008-10-13,12.32,0.062069,0.066863,5.042738,5.109602,-2.437097,0.145853,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13465088,Zylog Systems Ltd.,2024-03-21,0.35,0.000000,0.018215,1.441807,1.460021,0.572866,1.270303,-0.026511,0.002663,0.002803,-0.002113,0.025498,0.008887,0.207118
13465089,Zylog Systems Ltd.,2024-03-22,0.35,0.000000,0.018215,0.593324,0.611538,0.752002,0.124527,-0.026262,0.002406,0.002773,-0.002086,0.024682,0.008058,0.220363
13465090,Zylog Systems Ltd.,2024-03-26,0.35,0.000000,0.072878,-0.044514,0.028364,-1.091597,0.440605,-0.026266,0.002386,0.002795,-0.002064,0.024686,0.008062,0.220298
13465091,Zylog Systems Ltd.,2024-03-27,0.35,0.000000,0.018215,0.326702,0.344916,-0.165072,-0.188505,-0.026598,0.002727,0.003744,-0.001882,0.035403,0.018961,0.095276


In [33]:
ols180.to_csv("ols180.csv")